## QuantAQ Data
Data are available via an API. 

The first step is to sign up for a free QuantAQ account: https://app.quant-aq.com/auth/sign-up

<img src="./images/QuantAQ-SignUp.png" alt="QuantAQ Signup" width="400"/>

The next step is to obtain an API Key:

From the API Documentation

Obtaining an API key
Authentication is handled using HTTP Basic Auth which consists of sending an API key that is unique to your account with each request. This key should be kept secret and not shared with anyone. This API key is tied to your user account and will only allow you to take actions within your assigned scope of permissions.

You can generate an API key for your account by visiting the Developer section within the QuantAQ Cloud and clicking Generate New Key in the upper right-hand corner of your screen. It is recommended that you set this key as an environment variable on your computer with the name QUANTAQ_APIKEY.

To include your API key in your request, add an HTTP Authorization header, consisting of a usernameand password. When using httpie, you can use the -aflag to indicate authorization. The API key serves as the username and the password should be left blank.

<img src="./images/QuantAQ-Key.png" alt="QuantAQ API-Key" width="800"/>

Install the python package available for working with the API which is fully described here: https://github.com/quant-aq/py-quantaq.git

In [ ]:
# install py-quantaq
!pip install git+https://github.com/quant-aq/py-quantaq.git

In [10]:
# import the library
import quantaq
import os
# Set the API key as an environment variable within the notebook session
os.environ['QUANTAQ_APIKEY'] = 'A2XUQ82NCPSQUIUYLA2QLX82'
# Initialize the QuantAQ client
client = quantaq.QuantAQAPIClient(api_key=os.environ['QUANTAQ_APIKEY'])

In [11]:
# verify account information
whoami = client.whoami()
print (whoami)

{'confirmed': True, 'email': 'km2505@columbia.edu', 'first_name': 'Kytt', 'id': 2703, 'is_administrator': False, 'last_name': 'MacManus', 'last_seen': '2025-01-30T20:24:55.112864', 'member_since': '2025-01-30T14:59:34.179734', 'role': 1, 'username': 'kyttmacmanus'}


In [12]:
# You can retrieve a list of all the organizations visible to you:

organizations = client.organizations.list()
print (organizations)

[{'created_on': '2025-01-30T14:59:34.186321+00:00', 'description': 'Only you can see devices here', 'devices': [], 'id': 2573, 'members': [{'role': 'admin', 'user': 'kyttmacmanus'}], 'name': 'kyttmacmanus Sandbox 2337', 'networks': [], 'sandbox': True}]
